# *PyThermoML* - Import Template
This template explains how *PyThermoML* can be used, to write data into ThermoML.

**WARNING API is still changig. Only use under reservation.**


Before usage make sure that you've installed json, pydantic and lxml e. g. via pip.

In [1]:
from pythermo.thermoml.core import PureOrMixtureData, DataReport, Compound, DataPoint

from pythermo.thermoml.vars.componentcomposition import ComponentCompositionBase
from pythermo.thermoml.vars.temperature import TemperatureBase
from pythermo.thermoml.props.transportproperties import TransportProperty

from pythermo.thermoml.props.volumetricproperties import VolumetricProperty
from pythermo.thermoml.tools.writeTools import ThermoMLWriter
from pythermo.thermoml.tools.readTools import ThermoMLReader
from pydantic.json import pydantic_encoder

## DataReport
First create a dataReport instance, in which you can declare the title, the DOI and the authors of the referred paper.


In [2]:

# title, DOI, authors
authors = {
    "author 1": "authror1",
    "author 2": "author2",
    "author 3": "author3"
}

dataReport = DataReport(title="Title of referred",
                        DOI="Doi of referred paper", authors=authors)

## Compounds
After that you can declare the compounds which you have used in your chemical system.

In [3]:
comp1 = Compound(ID="c1", standardInchI="inhi1", standardInchIKey="inchikey1", smiles="smiles1", commonName="water")
comp2 = Compound(ID="c2", standardInchI="inchi2", standardInchIKey="inchikey2", smiles="smiles2", commonName="ethanol")

The attributes of Compounds are an internal ID, the inchi code, inchi key, smiles code and a common Name. In the next step, add the compounds to dataReport. With the ID you can reference to the compound later. IDs have to start with the letter 'c'.

In [4]:
comp1_ID = dataReport.addCompound(comp1)
comp2_ID = dataReport.addCompound(comp2)

## Pure or mixture data
The main part begins here: Declare in a PureOrMixtureData object which compounds you have used, by putting them into an array and calling them by their ID. Give your experiment also an ID that has to start with the letters 'pom'.

In [5]:
comps = [comp1_ID, comp2_ID]
experiment = PureOrMixtureData(ID="pom1", comps=comps)

## Declaration of Property and Variable
Declare your examined property (e.g. the viscosity) and variables (e. g. the temperature, mole fraction) of the experiment/simulation.

Concerning the property, enter again an ID (begins with letter 'p') and indicate whether you've done a simulation or an experiment.
The variable has also an ID (begins with letter 'v'). Be aware, that there are compound specific variables/properties (e. g. mole fraction). The function returns an ID to refer to respective property/variable.

In [6]:
dens = VolumetricProperty.massDensity(ID='p1', method='simulation')
sdiffCoeff1 = TransportProperty.selfDiffusionCoefficient(
    ID="p2", method='simulation', compoundID=comp1_ID)
sdiffCoeff2 = TransportProperty.selfDiffusionCoefficient(
    ID="p3", method='simulation', compoundID=comp2_ID)

# Variable definitions
temp = TemperatureBase.temperature(ID="v1")

frac1 = ComponentCompositionBase.moleFraction('v2', comp1_ID)
frac2 = ComponentCompositionBase.moleFraction('v3', comp2_ID)

This API provides the following properties:
* volumetric properties:
    * mass density (kg/m3)
* heat capacity properties:
    * molar heat capacity at constant pressure (J/K/mol)
    * molar heat capacity at constant volume (J/K/mol)
* transport properties:
    * viscosity (Pa*s)
    * kinematic viscosity (m2/s)
    * microviscosity (Pa*s)
    * self dffusion coefficient (m2/s)
* bioproperties:
    * peakTemperature (K)
* other:
    * surface tension (N/m) 
    * speed of sound (m/s)

This API provides the following variables:
* component composition:
    * mole fraction ()
* temperatures:
    * temperature(K)
    * upper temperature(K)
    * lower temperature(K)
* pressure:
    * pressure (kPa)
    

The units are fixed and cannot be changed!
For usage import the respective packages.

Add initialized properties/variables to experiement. The experiment is now ready to get filled with data.


In [7]:
densID = experiment.addProperty(dens)
dffCoeff1ID = experiment.addProperty(sdiffCoeff1)
dffCoeff2ID = experiment.addProperty(sdiffCoeff2)
tempID = experiment.addVariable(temp)
frac1ID = experiment.addVariable(frac1)
frac2ID = experiment.addVariable(frac2)

## Add data to experiment
Data can be added to experiment by creating Datapoints. Each Datapoint needs an identifier on which measurement you refer to (has to begin with 'meas'), the measured value, the property/variable used and an uncertainty of the measured value.

In [8]:
measurementID = "meas1"

viscDataPoint = DataPoint(
    measurementID=measurementID,
    value=10.0,
    propID=densID,
)

sdiff1DataPoint1 = DataPoint(
    measurementID=measurementID,
    value=10334,
    propID=dffCoeff1ID
)

sdiff2DataPoint1 = DataPoint(
    measurementID=measurementID,
    value=123123,
    propID=dffCoeff2ID
)

tempDataPoint = DataPoint(
    measurementID=measurementID,
    value=300.0,
    varID=tempID,
)

frac1DataPoint = DataPoint(
    measurementID=measurementID,
    value=0.2,
    varID=frac1ID,
)

frac2DataPoint = DataPoint(
    measurementID=measurementID,
    value=0.8,
    varID=frac2ID,
)

measurementID = "meas2"
sdiff1DataPoint2 = DataPoint(
    measurementID=measurementID,
    value=10334,
    propID=dffCoeff1ID
)

sdiff2DataPoint2 = DataPoint(
    measurementID=measurementID,
    value=123123,
    propID=dffCoeff2ID
)
viscDataPoint2 = DataPoint(
    measurementID=measurementID,
    value=1000.0,
    propID=densID,
    uncertainty=0.1
)

tempDataPoint2 = DataPoint(
    measurementID=measurementID,
    value=1000.0,
    varID=tempID,
    uncertainty=10.0
)

frac1DataPoint2 = DataPoint(
    measurementID=measurementID,
    value=1000.0,
    varID=frac1ID,
    uncertainty=0.01
)

frac2DataPoint2 = DataPoint(
    measurementID=measurementID,
    value=1000,
    varID=frac2ID,
    uncertainty=0.02
)

datapoints = [viscDataPoint, sdiff1DataPoint1, sdiff2DataPoint1,
              tempDataPoint, frac1DataPoint, frac2DataPoint]

datapoints2 = [viscDataPoint2, sdiff1DataPoint2, sdiff2DataPoint2, tempDataPoint2,
               frac1DataPoint2, frac2DataPoint2]

For each variable and property an uncertainty can be declared. For variables this API stores the **expanded uncertainty**. It is a quantity defining an interval about the result of a measurement that may be expected to encompass a large fraction of the distribution of values that could reasonably be attributed to the measurand. For properties the API stores the **combined standard uncertainty**. Note: the chosen values dont have any chemical meaning.

In [ ]:
# add Measurement to experiment
experiment.addMeasurement(dataPoints=datapoints)
experiment.addMeasurement(dataPoints=datapoints2)

# add experiment to dataReport
dataReport.addPureOrMixtureData(experiment)

In [10]:
# pretty print dataReport object with functionalities provided by pydantic
print(dataReport.json(exclude_none=True, indent=4))

{
    "title": "Title of referred",
    "DOI": "Doi of referred paper",
    "authors": {
        "author 1": "authror1",
        "author 2": "author2",
        "author 3": "author3"
    },
    "compounds": {
        "c1": {
            "ID": "c1",
            "standardInchI": "inhi1",
            "standardInchIKey": "inchikey1",
            "smiles": "smiles1",
            "commonName": "water"
        },
        "c2": {
            "ID": "c2",
            "standardInchI": "inchi2",
            "standardInchIKey": "inchikey2",
            "smiles": "smiles2",
            "commonName": "ethanol"
        }
    },
    "pureOrMixtureData": {
        "pom1": {
            "ID": "pom1",
            "comps": [
                "c1",
                "c2"
            ],
            "properties": {
                "p1": {
                    "propName": "Mass density",
                    "propGroup": "VolumetricProp",
                    "ID": "p1",
                    "unit": "kg/m3",
         

Congratulations your DataReport object has been created sucessfully. Now you can write your DataReport to a ThermoML file by using the writeThermo() function. If you want to modify your ThermoML file you can read it by readFromThermoMLFile()

In [11]:
# write to 'testThermo.xml'
writer = ThermoMLWriter(dataRep=dataReport, filename="testThermo.xml")
writer.writeThermo()

# read from 'testThermo.xml'
reader1 = ThermoMLReader(path="testThermo.xml")
dataRepRead1 = reader1.readFromThermoMLFile()

# reading from 'testThermo.json' also works
reader2 = ThermoMLReader(path="testThermo.json")
dataRepRead2 = reader2.readFromJSON()

# accesing data with softdata functionalities
dataRepRead2.getPureOrMixtureData("pom1").getMeasurement("meas1").getProperty("p1")

[<Element {http://www.iupac.org/namespaces/ThermoML}nVarValue at 0x1b6636aeb00>]
[<Element {http://www.iupac.org/namespaces/ThermoML}nVarValue at 0x1b6636aebc0>]
[<Element {http://www.iupac.org/namespaces/ThermoML}nVarValue at 0x1b6636aec00>]
[<Element {http://www.iupac.org/namespaces/ThermoML}nVarValue at 0x1b6636ae980>]
[<Element {http://www.iupac.org/namespaces/ThermoML}nVarValue at 0x1b6636b5180>]
[<Element {http://www.iupac.org/namespaces/ThermoML}nVarValue at 0x1b6636b5240>]


DataPoint(measurementID='meas1', value=10.0, propID='p1', varID=None, uncertainty=None, numberOfDigits=None, data_point_type='Property', elementID='p1')